In [1]:
import pandas as pd
import numpy as np
from pyswarm import pso
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

In [2]:
#需要将神经网络模型文件，和数据表放到此py文件目录下
# 读取Excel文件
excel_file1 = "1117_load.xls"  # 负荷预测输入数据集
excel_file2 = "1117_temp.xls"  # 温度预测输入数据集，设定温度也在这里面
excel_file3 = "1117_weather.xls"  # 气象预报输入数据集
dataset1 = pd.read_excel(excel_file1, usecols="A:E", header=None)
dataset2 = pd.read_excel(excel_file2, usecols="A:H", header=None)
data3 = np.array(pd.read_excel(excel_file3, usecols="A", header=None))

In [3]:
# 数据归一化处理
scaler = MinMaxScaler()
data1_scaled = scaler.fit_transform(dataset1)
data2_scaled = scaler.fit_transform(dataset2)
# data3_scaled = scaler.fit_transform(dataset3)

In [4]:
#参数
batchsize = 1 #批处理大小，预测模型里是32
step = 5 #时间步数
loaddim = 5 #负荷预测模型输入维度
tempdim = 7 #温度预测模型输入维度
rows_to_predict = 10 #进行预测的迭代次数，序列长度为step，步长为1
nextTin = 0 #中间变量Tin(t+1)，具体数值不重要
Beta = 0.000008 #可变参数β
Alpha = 1-Beta #可变参数α

T_max, T_min = 22.89888889, 22.69222222 # Tin训练集 max=22.89888889，min=22.69222222
C_max, C_min = 1193.96, 1153.82 # Cooling训练集 max=1193.96，min=1153.82
nextTset = 23

In [5]:
# 加载负荷预测模型
loadmodel = tf.keras.models.load_model('1117_loadpred.keras')
# 加载温度预测模型
tempmodel = tf.keras.models.load_model('1117_temppred.keras')

In [6]:
################## 热泵能耗方程：##################
# 1号热泵：磁悬浮1
def PphCal1(Cooling1, nextTreturn, nextTout):
    return Cooling1 / (0.114064992 * Cooling1
                      + 9.86011205 * nextTreturn
                      - 0.394321578 * nextTout
                      + 0.0000450464015 * Cooling1 * Cooling1
                      - 0.00581368122 * Cooling1 * nextTreturn
                      - 0.000758651811 * Cooling1 * nextTout
                      - 0.326362421 * nextTreturn * nextTreturn
                     + 0.0184361005* nextTreturn * nextTout
                     + 0.00239640342 * nextTout * nextTout - 71.6350077532962)
power1 = PphCal1(173.34, 14.55, 37.91)
print(power1)

# 2号热泵：磁悬浮2
def PphCal2(Cooling2, nextTreturn, nextTout):
    return Cooling2 / ( - 0.031207272 * Cooling2
                      - 5.57093763 * nextTreturn
                      - 0.178609075 * nextTout
                      - 0.000140286197 * Cooling2 * Cooling2
                      + 0.0081496977 * Cooling2 * nextTreturn
                      - 0.000599404697 * Cooling2 * nextTout
                      + 0.150016223 * nextTreturn * nextTreturn
                     - 0.00555278723 * nextTreturn * nextTout
                     + 0.00426582024 * nextTout * nextTout + 48.815621285422)
power2 = PphCal2(191.78, 14.59, 37.91)
print(power2)

# 3号热泵：螺杆1

def PphCal3(Cooling3, nextTreturn, nextTout):
    return Cooling3 / (- 0.0565696352 * Cooling3
                      + 140.258199 * nextTreturn
                      + 0.297813736 * nextTout
                      - 0.000000512788461 * Cooling3 * Cooling3
                      + 0.00512409975 * Cooling3 * nextTreturn
                      - 0.000396810401 * Cooling3 * nextTout
                      - 5.00822474 * nextTreturn * nextTreturn
                     - 0.00851263147* nextTreturn * nextTout
                     - 0.000209590663 * nextTout * nextTout -986.272338515734)
'''
def PphCal3(Cooling3, nextTreturn, nextTout):
    return Cooling3 / (0.0108998754 * Cooling3
                      - 69.4648733 * nextTreturn
                      - 0.382184169 * nextTout
                      + 0.0000107828275 * Cooling3 * Cooling3
                      - 0.000685155745 * Cooling3 * nextTreturn
                      - 0.000287410665 * Cooling3 * nextTout
                      + 2.3373995 * nextTreturn * nextTreturn
                      + 0.0342744052* nextTreturn * nextTout
                      - 0.000282931457 * nextTout * nextTout + 516.111737024876)
'''                      
power3 = PphCal3(646.4, 14.31, 37.91)
print(power3)


# 4号热泵：螺杆2
def PphCal4(Cooling4, nextTreturn, nextTout):
    return Cooling4 / (- 0.0316576759 * Cooling4
                      - 9.98926426 * nextTreturn
                      + 0.0902543953 * nextTout
                      + 0.0000000479863189 * Cooling4 * Cooling4
                      + 0.00257982592 * Cooling4 * nextTreturn
                      - 0.0000778893844 * Cooling4 * nextTout
                      + 0.295875914 * nextTreturn * nextTreturn
                     - 0.00932111852 * nextTreturn * nextTout
                     + 0.000430676823 * nextTout * nextTout + 84.2195052674394)
power4 = PphCal4(587.45, 14.58, 37.91)
print(power4)

60.19008805479266
62.70218705354646
256.330379316575
274.29993549338934


In [7]:
################## 水泵能耗方程：##################
# Pwp=0.0322*Volume*Volume-1.7627*Volume+26.2738
def PwpCal(Volume):
    return 0.0238* Volume* Volume - 5.27 * Volume + 306.7
pump = PwpCal(120)
print(pump)

17.02000000000004


In [8]:
################## 负荷能量守恒方程：##################
# Cooling = (Treturn - Tsupply)* Volume*4.2/3.6
# 其中，
# 45=< Volume =<90
# 9=< Tsupply =<15
# 12.5=< Treturn =<16.5
# 且Tsupply < Treturn
def Constraint1(x, Cooling):
#   Cooling = 2 * (Cooling_screw + Cooling_maglev)
    nextTsupply, nextTreturn, nextVscrew, nextVmaglev= x
    return (Cooling == (nextTreturn - nextTsupply) * 2 * (nextVscrew + nextVmaglev) * 4.2 / 3.6) and nextTsupply < nextTreturn
# 如果有多个Volume，可以分别设置，但是在计算Cooling时要加和

In [9]:
################## 代价函数：##################
# J=α*(Tset-Tin)2+β(Php+Pwp)2
# 其中，α=0.5，β=0.5（α，β赋值后续需要调整，暂定均为0.5）
def Cost(x, Alpha, Beta, nextTset, nextTout, Cooling):
    nextTsupply, nextTreturn, nextVscrew, nextVmaglev= x
    Pwp = PwpCal(2 / 3 * (nextVscrew + nextVmaglev))
    Cooling_maglev = Cooling / 2 * nextVmaglev / (nextVscrew + nextVmaglev)
    Cooling_screw = Cooling / 2 - Cooling_maglev
    Php1 = PphCal1(Cooling_maglev,nextTreturn,nextTout)
    Php2 = PphCal2(Cooling_maglev, nextTreturn, nextTout)
    Php3 = PphCal4(Cooling_screw, nextTreturn, nextTout)
    Php4 = PphCal4(Cooling_screw, nextTreturn, nextTout)
    # tempdata = np.append(KnownParams, nextTsupply)
    # tempdata = np.insert(KnownParams, 6, nextTsupply, axis= 1) # 温度预测比负荷预测多一个参数，给加进去
    #[1,5,7]
    # tempinput = np.tile(tempdata, (5,1)).reshape((batchsize, step, tempdim))
    # tempdata = data2[: , 0:7]
    tempinput = tempdata.reshape((batchsize, step, tempdim)) # 二维数组按照神经网络模型reshape
    global nextTin  # 更新nextTin的值
    global nextTin_scaled  # 更新nextTin的值
    nextTin_scaled = tempmodel.predict(tempinput,verbose=0)[0][0] # 计算出来的nextTin是高维张量，需要取其中的元素作为值
    nextTin = nextTin_scaled * (T_max - T_min) + T_min #温度预测值反归一化
    cost = Alpha * (nextTset - nextTin)*(nextTset - nextTin) + Beta * (Php1 + Php2+ Php3+ Php4 + Pwp * 3) *  (Php1 + Php2+ Php3+ Php4 + Pwp * 3)
    # cost = Alpha * (23 - nextTin)*(23 - nextTin) + Beta * (Php1 + Php2+ Php3+ Php4 + Pwp * 3) *  (Php1 + Php2+ Php3+ Php4 + Pwp * 3)
    return cost
'''
def Tin_deNormalize(nextTin_scaled, data2):
    Tins =  data2[:, 2]
    T_max, T_min = max(Tins), min(Tins)
    nextTin = nextTin_scaled * (T_max - T_min) + T_min
    return nextTin
def Co_deNormalize(Cooling_scaled, data1):
    coolings = data1[:, 0]
    c_max, c_min = max(coolings), min(coolings)
    Cooling = Cooling_scaled * (c_max - c_min) + c_min
    return Cooling
'''

'\ndef Tin_deNormalize(nextTin_scaled, data2):\n    Tins =  data2[:, 2]\n    T_max, T_min = max(Tins), min(Tins)\n    nextTin = nextTin_scaled * (T_max - T_min) + T_min\n    return nextTin\ndef Co_deNormalize(Cooling_scaled, data1):\n    coolings = data1[:, 0]\n    c_max, c_min = max(coolings), min(coolings)\n    Cooling = Cooling_scaled * (c_max - c_min) + c_min\n    return Cooling\n'

In [10]:
if __name__ == '__main__':
    # tempred.summary()
    ####待求解参数####
    # x = [nextTsupply,nextTreturn,nextVolume]
    x = []
    #####待优化函数值####
    # f = cost()
    cf = []
    co = []
    tin = []
    sc1 = []
    ma1 = []
    sc2 = []
    ma2 = []
    pu =[]
    tot = []
    deltaT = []  # 新增：存储供回水温差
    # 迭代预测过程，已将本轮Cooling，Tin(t+1)应用到下轮预测中
    for i in range(rows_to_predict):
        print('滚动次数：', i)
        tempdata = data2_scaled[i:i+5, 0:7]
        loaddata = data1_scaled[i:i+5, 0:5]
        loadinput = loaddata.reshape((batchsize,step,loaddim))
        #[1,5,6]
        Cooling_scaled = loadmodel.predict(loadinput,verbose=0)[0][0]
        #Cooling = Co_deNormalize(Cooling_scaled, data1)
        Cooling = Cooling_scaled * (C_max - C_min) + C_min #负荷预测值反归一化
        nextTout = data3[i+5,0]
        # nextTset = np.array(dataset2)[i+5,7]
        lb = [11, 14.5, 110, 35] # 流量lower bound必须大于0，不然在负荷分配中不能作为分母
        ub = [12, 15.5, 180, 70] # nextTsupply, nextTreturn, nextVscrew, nextVmaglev
        Constraint_with_param = lambda x: Constraint1(x, Cooling)
        Cost_with_param = lambda x: Cost(x, Alpha, Beta, nextTset, nextTout, Cooling)
        xopt,fopt = pso(Cost_with_param, lb, ub, f_ieqcons=Constraint_with_param, minfunc=0.1, maxiter=50)
        #更新Cooling，Tin(t+1)
        data2_scaled[i+5,1] = Cooling_scaled*0.6 + data2_scaled[i+5,1]*0.4 # 更新温度预测输入参数中的t时刻冷量
        data2_scaled[i+6,6] = Cooling_scaled*0.6 + data2_scaled[i+6,6]*0.4 # 更新温度预测输入参数中的t-1时刻冷量
        data1_scaled[i+5,0] = Cooling_scaled*0.6 + data1_scaled[i+5,0]*0.4 # 更新负荷预测输入参数中的t时刻冷量
        data1_scaled[i+5,1] = nextTin_scaled*0.4 + data1_scaled[i+5,1]*0.6 # 更新负荷预测输入参数中的t时刻温度
        data2_scaled[i+5,2] = nextTin_scaled*0.4 + data2_scaled[i+5,2]*0.6 # 更新温度预测输入参数中的t时刻温度
        print('更新后的Cooling：', data2_scaled[i+5,1])
        cooling_maglev = Cooling / 2 * xopt[3] / (xopt[2] + xopt[3])
        cooling_screw =  Cooling / 2  - cooling_maglev
        volume_pump = 2 / 3 * (xopt[2] + xopt[3])
        power_screw1 = PphCal4(cooling_screw, xopt[1], nextTout)
        power_screw2 = PphCal4(cooling_screw, xopt[1], nextTout)
        power_maglev1 = PphCal1(cooling_maglev, xopt[1], nextTout)
        power_maglev2 = PphCal2(cooling_maglev, xopt[1], nextTout)
        power_pump = PwpCal(volume_pump)
        power_total = ( power_screw1 + power_maglev1 + power_screw2 + power_maglev2) + 3 * power_pump
        # 新增：计算供回水温差
        deltaT_current = xopt[1] - xopt[0]
        deltaT.append(deltaT_current)
        print('供温，回温，螺杆流量，磁悬浮流量:',xopt)
        print('供回水温差:', deltaT_current)  # 新增：打印供回水温差
        print('Cost:',fopt)
        print('总冷量：', Cooling)
        print('室温：', nextTin)
        print('螺杆功率：', power_screw1, power_screw2)
        print('磁悬浮功率：', power_maglev1, power_maglev2)
        print('水泵功率：', power_pump)
        print('总功率：', power_total)
        x.append(xopt)
        cf.append(fopt)
        co.append(Cooling)
        tin.append(nextTin)
        sc1.append(power_screw1)
        sc2.append(power_screw2)
        ma1.append(power_maglev1)
        ma2.append(power_maglev2)
        pu.append(power_pump)
        tot.append(power_total)
    print('最终优化结果：')
    print('Tsupply(t+1)，Treturn(t+1),Volume(t+1):',x)
    print('代价函数最优值：',cf)
    print('冷量变化：', co)
    print('室温：', tin)
    print('供回水温差变化：', deltaT)  # 新增：打印供回水温差变化

滚动次数： 0
Stopping search: Swarm best objective change less than 0.1
更新后的Cooling： 0.41370043458187955
供温，回温，螺杆流量，磁悬浮流量: [ 11.4283606   14.50063856 110.          64.48543037]
供回水温差: 3.0722779551039388
Cost: 2.0826880167100237
总冷量： 1168.0832257401944
室温： 22.790162760105268
螺杆功率： 182.8707297222994 182.8707297222994
磁悬浮功率： 42.53154548090515 49.38581038776665
水泵功率： 15.716715429873886
总功率： 504.8089616028922
滚动次数： 1
Stopping search: Swarm best objective change less than 0.1
更新后的Cooling： 0.46744450106214686
供温，回温，螺杆流量，磁悬浮流量: [ 11.2129626   14.5        110.65434249  57.29759279]
供回水温差: 3.2870374027489113
Cost: 2.0944965094873313
总冷量： 1170.2120371210574
室温： 22.78499894380837
螺杆功率： 185.13954450505466 185.13954450505466
磁悬浮功率： 43.01680874404277 47.68626476134014
水泵功率： 15.00526339626532
总功率： 505.99795270428814
滚动次数： 2
Stopping search: Swarm best objective change less than 0.1
更新后的Cooling： 0.4283673700077003
供温，回温，螺杆流量，磁悬浮流量: [ 11.26519754  14.5        110.71534291  50.85103161]
供回水温差: 3.2348024589388

Stopping search: Swarm best objective change less than 0.1
更新后的Cooling： 0.30313081046094226
供温，回温，螺杆流量，磁悬浮流量: [ 11.42571791  14.5        121.06651494  55.43827559]
Cost: 2.033587325960405
总冷量： 1166.3927845531703
室温： 22.73569124490235
螺杆功率： 180.58395799069277 180.58395799069277
磁悬浮功率： 41.478610196749024 44.44092009300594
水泵功率： 16.119301584358936
总功率： 495.4453510242173
滚动次数： 155
Stopping search: Swarm best objective change less than 0.1
更新后的Cooling： 0.2411738195523867
供温，回温，螺杆流量，磁悬浮流量: [ 11.15665864  14.5        110.          58.5323537 ]
Cost: 1.957206104640087
总冷量： 1164.5811951947212
室温： 22.732654289755054
螺杆功率： 177.18557832156557 177.18557832156557
磁悬浮功率： 40.79772538858692 45.24192324585868
水泵功率： 15.031917778201318
总功率： 485.5065586121807
滚动次数： 156
Stopping search: Swarm best objective change less than 0.1
更新后的Cooling： 0.21110560027116917
供温，回温，螺杆流量，磁悬浮流量: [ 11.65701071  14.5        110.          67.99552947]
Cost: 1.942390212078446
总冷量： 1162.522964658141
室温： 22.727441745368836
螺杆功率： 1

Stopping search: Swarm best objective change less than 0.1
更新后的Cooling： 0.4338070182047147
供温，回温，螺杆流量，磁悬浮流量: [ 12.          14.5        111.50125128  63.81848113]
Cost: 2.172863190917145
总冷量： 1171.0750228512286
室温： 22.869334051978548
螺杆功率： 188.28287392276687 188.28287392276687
磁悬浮功率： 44.04581205286734 50.87926039356965
水泵功率： 15.872586364528388
总功率： 519.1085793855559
滚动次数： 177
Stopping search: Swarm best objective change less than 0.1
更新后的Cooling： 0.4295277560654128
供温，回温，螺杆流量，磁悬浮流量: [ 11.97045166  14.50748343 110.          58.31996121]
Cost: 2.1987203872192422
总冷量： 1170.2087402141094
室温： 22.871902305298704
螺杆功率： 191.20362364917315 191.20362364917315
磁悬浮功率： 44.73062156641647 50.09012312526602
水泵功率： 15.021337319302575
总功率： 522.2920039479366
滚动次数： 178
Stopping search: Swarm best objective change less than 0.1
更新后的Cooling： 0.4088180265114218
供温，回温，螺杆流量，磁悬浮流量: [ 11.98648614  14.58426122 110.          64.55220867]
Cost: 2.293507703738783
总冷量： 1169.5565926402808
室温： 22.87551466949017
螺杆功率： 19

Stopping search: Swarm best objective change less than 0.1
更新后的Cooling： 0.4099446639262695
供温，回温，螺杆流量，磁悬浮流量: [ 12.          14.5        120.23958908  48.37115932]
Cost: 1.9618120661069272
总冷量： 1168.5052980166674
室温： 22.776910857033112
螺杆功率： 179.83723108582163 179.83723108582163
磁悬浮功率： 40.84020941974206 43.25898487657567
水泵功率： 15.036064189878402
总功率： 488.88184903759617
滚动次数： 199
Stopping search: Swarm best objective change less than 0.1
更新后的Cooling： 0.35374482765563753
供温，回温，螺杆流量，磁悬浮流量: [ 11.          14.5        131.48096969  35.        ]
Cost: 2.073702610051401
总冷量： 1166.2388623034954
室温： 22.77223043520787
螺杆功率： 184.30073283801647 184.30073283801647
磁悬浮功率： 40.11769983342019 49.09254523782918
水泵功率： 14.969631289315146
总功率： 502.7206046152278
滚动次数： 200
Stopping search: Swarm best objective change less than 0.1
更新后的Cooling： 0.33268088633050896
供温，回温，螺杆流量，磁悬浮流量: [ 11.0762643   14.51417547 110.          57.29436006]
Cost: 1.8633895723885703
总冷量： 1164.0296846288443
室温： 22.763103442040286
螺杆功率

In [11]:
water1 = [arr[0] for arr in x]
water2 = [arr[1] for arr in x]
water3 = [arr[2] for arr in x]
water4 = [arr[3] for arr in x]

result = {'Supply': water1, 'Return': water2, 'Vscrew': water3, 'Vmaglev': water4,
              'Cooling': co, 'Temp': tin, 'Pscrew1': sc1, 'Pscrew2': sc2, 'Pmaglev1': ma1, 'Pmaglev2': ma2, 'Ppump': pu,
              'Total_power': tot}
re = pd.DataFrame(result)
print(re)
#re.to_csv('1121_GZZ_output1.csv', index=False)

      Supply     Return      Vscrew    Vmaglev      Cooling       Temp  \
0  11.428361  14.500639  110.000000  64.485430  1168.083226  22.790163   
1  11.212963  14.500000  110.654342  57.297593  1170.212037  22.784999   
2  11.265198  14.500000  110.715343  50.851032  1169.831110  22.780175   
3  11.447068  14.500000  110.000000  65.335737  1169.831663  22.776102   
4  11.003295  14.500000  115.376086  66.171600  1171.589517  22.765262   
5  11.013309  14.500000  110.000000  70.000000  1172.594773  22.760353   
6  11.368287  14.500000  110.000000  64.169227  1172.620009  22.752791   
7  11.053494  14.500000  110.000000  69.500952  1171.918474  22.743453   
8  11.947228  14.500000  110.478215  70.000000  1170.588916  22.736756   
9  11.107273  14.559990  110.000000  68.747771  1168.345725  22.738457   

      Pscrew1     Pscrew2   Pmaglev1   Pmaglev2      Ppump  Total_power  
0  182.870730  182.870730  42.531545  49.385810  15.716715   504.808962  
1  185.139545  185.139545  43.016809 